## Approach Vector Store

In [ ]:
!pip install torch torchvision faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 56.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.1/78.2 GB disk)


In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import cv2

# Load YOLOv5 model
yolo_model = YOLO("yolov8s.pt")

model = models.resnet50(pretrained=True)
model.eval()

# Define a transformation pipeline
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def detect_and_crop(image_path):
    image = Image.open(image_path)
    image_np = np.array(image)

    # Detect objects in the image
    results = yolo_model(image_np)
    boxes = results[0].boxes.xyxy  # Get bounding boxes

    if len(boxes) > 0:
        # Take the first detected object for cropping
        x1, y1, x2, y2 = map(int, boxes[0])
        image = image.crop((x1, y1, x2, y2))

    return image

def extract_features(image_path):
    image = detect_and_crop(image_path)
    image = preprocess(image)
    image = image.unsqueeze(0)

    with torch.no_grad():
        features = model(image)

    return features.numpy()

In [ ]:
import faiss

image_paths = ['/content/Data analyst operation-MuhFaridanSutariya.jpg', '/content/messi 1.jpeg', '/content/foto mantap.jpg', '/content/209105611_503055027473834_75509644885757551_n.jpg']

feature_vectors = [extract_features(image_path) for image_path in image_paths]

features = np.vstack(feature_vectors).astype(np.float32)

# Create FAISS index
d = features.shape[1]  # dimension of feature vectors
index = faiss.IndexFlatL2(d)

index.add(features)

# Save the index
faiss.write_index(index, 'image_index.faiss')


0: 640x480 1 person, 86.8ms
Speed: 33.0ms preprocess, 86.8ms inference, 711.9ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 person, 12.3ms
Speed: 4.5ms preprocess, 12.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 480)

0: 640x640 1 person, 2 dining tables, 1 cell phone, 17.3ms
Speed: 4.4ms preprocess, 17.3ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 16.8ms
Speed: 4.4ms preprocess, 16.8ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
def search_similar_images(query_image_path, index, k=1):
    query_features = extract_features(query_image_path).astype(np.float32)

    D, I = index.search(query_features, k)

    return I

index = faiss.read_index('image_index.faiss')

query_image_path = '/content/2111130 MUHAMMAD FARIDAN SUTARIYA.JPG'

similar_image_indices = search_similar_images(query_image_path, index)

print(similar_image_indices)


0: 448x640 1 person, 64.1ms
Speed: 4.8ms preprocess, 64.1ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)
[[0]]


In [ ]:
similar_image_paths = [image_paths[i] for i in similar_image_indices[0]]

print("Similar images:")
for path in similar_image_paths:
    print(path)

Similar images:
/content/Data analyst operation-MuhFaridanSutariya.jpg
